<a href="https://colab.research.google.com/github/HwangHanJae/Recommentaion_Study/blob/main/book_Recommender_Systems/Nearest_Neighbor_Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이웃 기반 협업 필터링
| 메모리 기반 알고리즘

- 협업필터링의 가장 초기 알고리즘

**유사한 사용자들은 평점을 주는 방식에서 유사한 패턴을 보이고 유사한 아이템에는 유사한 평점을 주는 사실을 기반으로 함**





## 사용자 기반 협업 필터링

사용자A 에게 추천을 하기 위하여 사용자A와 유사한 사용자들의 평점을 이용
- 예측된 사용자A의 평점은 피어그룹의 각 아이템의 평점에 대한 가중 평균으로 산정
  - 가중평균 : 산술평균을 계산할 때 자료 값의 중요도나 영향에 따라 가중치를 반영하여 구한 평균값( 출처 : [위키백과](https://ko.wikipedia.org/wiki/%EA%B0%80%EC%A4%91_%EC%82%B0%EC%88%A0_%ED%8F%89%EA%B7%A0]))

### 사용자 기반 이웃 모델
- 평점 예측이 계산되는 타깃 유저와 유사한 사용자를 찾기 위하여 정의
- 타깃 유저 i의 이웃을 찾기 위하여 모든 사용자와의 유사도를 계산한다.
  - 유사도 계산은 사용자들마다 평점을 내리는 방식이 다를 수 있기 때문에 까다롭다.
    - 사용자1은 대부분의 상품을 좋아하는 성향이라 평점을 높게 매길 수 있음
    - 사용자2는 대부분의 상품을 싫어하는 성향이라 평점을 낮게 매길 수 있음
    - 서로 다른 사용자들은 같은 상품에 대해서만 평가하지 않음

위의 문제를 해결하기 위하여 아래의 매커니즘을 확인해야 한다. 


In [60]:
# 라이브러리
import pandas as pd
import numpy as np
import scipy.stats as stats

아래는 $m$ 사용자 $n$ 상품의 $m\times{n}$ 평점 행렬($R=[r_{uj}]$)를 정의한것이다.

In [77]:
#예제 테이블(R) 정의

values = [[7,6,7,4,5,4],
         [6,7,None,4,3,4],
         [None, 3,3,1,1,None],
         [1,2,2,3,3,4],
         [1,None,1,2,3,3]]

R = pd.DataFrame(values, columns = [1,2,3,4,5,6], index = [1,2,3,4,5])

R

,1,2,3,4,5,6
1,7.0,6.0,7.0,4,5,4.0
2,6.0,7.0,NaN,4,3,4.0
3,NaN,3.0,3.0,1,1,NaN
4,1.0,2.0,2.0,3,3,4.0
5,1.0,NaN,1.0,2,3,3.0


*$I_u$는 평점행렬($R$)에서 사용자($u$)에 의해 지정된 상품 색인 세트를 말한다.*

예를 들어 위의 테이블 R에서  
2번 사용자는 첫 번째(1), 두 번째(2), 네 번째(4), 다섯 번째(5), 여섯 번째(6) 아이템에 평점이 매겨져있고, 세 번째(3) 아이템에는 평점이 매겨져있지 않고 결측값(NaN)으로 표현되어 있을 때  
$I_2 = \{1,2,4,5,6\}$ 라 표현한다.

사용자(u), 사용자3(v) 모두가 평점을 매긴 아이템 집합은    
$I_{u}\cap{I_{v}}$라 표현한다.

In [78]:
# 사용자(u)가 평가한 아이템 집합을 반환하는 함수
# 평가하지 않았다면 None이기 때문에 인덱스를 반환하지 않음
def get_I_u(matrix, u):
  not_null_index = matrix.loc[u, matrix.loc[u].notnull()].index
  return set(not_null_index)

#I_2 2번 사용자가 평가한 아이템 집합
get_I_u(R, 2)

{1, 2, 4, 5, 6}

현실에서의 평점행렬(R)은 희소행렬의 형태로 평점을 매기지 않은 결측값(NaN)이 많이 분포되어 있을 것이다.

따라서 $I_{u}\cap{I_{v}}$는 공집합일수 있고 공집합인 경우도 많다.

두 사용자(u, v)의 유사도를 알아내는 방법은 아래와 같이 [피어슨 상관계수](https://ko.wikipedia.org/wiki/%ED%94%BC%EC%96%B4%EC%8A%A8_%EC%83%81%EA%B4%80_%EA%B3%84%EC%88%98)를 이용하는 방법이다.
- 피어슨 상관계수 -1 ~ 1의 사이의 값을 가짐
  - +1은 완벽한 양의 선형 상관 관계
  - -1은 완벽한 음의 선형 상관 관계
  - 0은 선형 상관 관계 없음
- 여기서는 1과 가까울수록 유사하다고 판단


In [80]:
#함수를 직접 만들어 유사도 계산

#유저의 평균 계산
def get_M_u(matrix, u):
  
  # I_u = get_I_u(user_id)
  # I_u_norm = len(I_u)

  # ruk = table.loc[user_id].sum()

  # M_u = ruk / I_u_norm
  M_u = matrix.loc[u].mean()
  return M_u


# 피어슨 상관계수
# 공분산 / 표준편차(u) * 표준편차(v)
def Pearson(matrix, u, v):

  I_u = get_I_u(matrix, u)
  I_v = get_I_u(matrix, v)
  k = I_u.intersection(I_v)
  M_u = get_M_u(matrix, u)
  M_v = get_M_u(matrix, v)

  u_std = np.sqrt(np.sum(np.power((matrix.loc[u, k] - M_u), 2)))
  v_std = np.sqrt(np.sum(np.power((matrix.loc[v, k] - M_v),2)))

  co = np.sum((matrix.loc[u, k] - M_u) * (matrix.loc[v, k] - M_v))
  return co / (u_std * v_std)

In [82]:
#1번, 3번 사용자의 유사도
Pearson(R, 1, 3)

0.8944271909999159

In [83]:
#1번, 3번 사용자의 유사도
#pandas의 corr()함수를 이용하여 유사도를 계산
pearson = R.T.corr(method='pearson')
pearson.loc[1,3]

0.8944271909999159

피어슨 계수는 타깃 사용자와 다른 모든 사용자 간의 계산이다.  
타깃 사용자의 피어 그룹을 정의하는 한 가지 방법으로는 타깃과 피어슨 계수가 가장 높은 k사용자 집합을 이용하는 것이다.
- 이때 k는 Pearson()함수에서 사용된 k와 다름

위에서 언급한 것 처럼 사용자들은 평점을 내리는 방식이 다를 수 있다.
- 책에서는 평점에 각기 다른 스케일을 부여한다고 나와있음

평점 그 자체는 피어 그룹의 평균 평점을 결정하기 전에 행에 대해서 평균을 중심으로 재배열되어야 한다.  

사용자 $u$의 아이템 $j$에 대한 평균 중심 평점 $s_{uj}$은 초기 평점 $r_{uj}$에서 사용자의 평균 평점을 빼는 것으로 정의할 수 있다.

$$s_{uj}=r_{uj}-μ_{u}$$


타깃 사용자 $u$의 상위-$k$ 피어 그룹에서의 아이템에 대한 평균 평점의 가중 평균은 평균 중심 예측에 활용된다.

이 예측 뒤에는 타깃 사용자의 평균 평점이 더해져 타깃 사용자 $u$의 아이템 $j$에 대한 평점 $\hat{r}_{uj}$예측이 이루어진다.


$$\hat{r}_{uj} = μ_{u}+\frac{∑_{u\in{P_{u}(j)}}^{}Sim(u, v)⋅s_{uj}}{∑_{u\in{P_{u}(j)}}^{}|Sim(u, v)|}$$

$$= μ_{u}+\frac{∑_{u\in{P_{u}(j)}}^{}Sim(u, v)⋅(r_{uj}-μ_{u})}{∑_{u\in{P_{u}(j)}}^{}|Sim(u, v)|}$$

$P(j)$는 아이템 $j$에 대해 평점을 매긴 타깃 사용자 $u$의 가장 유사한 사용자 $k$명의 집합 1이라 생각해보면 타깃 사용자 $u$와 매우 낮은 혹은 음의 상관을 보이는 사용자는 제외시키기도 한다.

$P(j)$를 어떻게 설정하느냐에 따라 예측된 평점이 달라진다